<a href="https://colab.research.google.com/github/rajivsraman/aipi530project/blob/main/aipi530_startercode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition

In [8]:
# import the NICU data from the CSV file

nicu_data = pd.read_csv("https://raw.githubusercontent.com/rajivsraman/aipi530data/refs/heads/main/nicu_orders.csv")

# group the data into each week of deliveries and compute the total demand per each week

weekly_demand = nicu_data.groupby('Delivery Date')['Bottle Count'].sum().tolist()

# create the model - a concrete model will be sufficient for maximizing our objective function

model = ConcreteModel()

# generate the set of WEEKS - the number of weeks is determined from the list of dates given in the CSV

model.WEEKS = RangeSet(len(weekly_demand))

# define the decision variables

model.Output = Var(model.WEEKS, domain = NonNegativeIntegers) # number of milk bottles sent out to NICUs and other locations per week
model.Process = Var(model.WEEKS, domain = NonNegativeReals) # number of milk bottles processed from raw milk per week (mL)

# define the total weekly demand (units: number of bottles) as a parameter

model.demand = {w: weekly_demand[w-1] for w in model.WEEKS}

# define the variables for tracking different volumes of milk

model.iRV = Var(model.WEEKS, domain = NonNegativeReals) # raw volume at the start of week w
model.fRV = Var(model.WEEKS, domain = NonNegativeReals) # raw volume at the end of week w
model.iBV = Var(model.WEEKS, domain = NonNegativeReals) # bottled volume at the start of week w
model.fBV = Var(model.WEEKS, domain = NonNegativeReals) # bottled volume at the end of week w
model.fTV = Var(model.WEEKS, domain = NonNegativeReals) # total stored volume at the end of week w

# set the constants associated with our optimization problem

init_raw_vol = 15000 # initial volume of raw milk before any donations (mL)
init_bot_vol = 20000 # initial volume of bottled milk before any donations (mL)
raw_donation = 35000 # weekly volume of raw milk donated to the inventory (mL)
process_cap = 40000 # the maximum volume of raw milk that can be processed to bottled milk within a week (mL)
contaminated_frac = 0.05 # the approximate fraction of processed milk that is thrown out each week due to contamination
bottle_vol = 100 # the volume of processed milk in one bottle (mL)
min_stored_bottles = 50 # the minimum number of processed milk bottles that must be kept in the milk bank inventory

# initialize the proper values for week 1 to make sure the model starts correctly

model.iRV[1].fix(init_raw_vol) # initialize the correct starting raw volume
model.iBV[1].fix(init_bot_vol) # initialize the correct starting bottled volume

# constraint 1. the final raw volume of each week is equal to the starting raw volume of that week minus the volume processed during that week

def Raw_Volume_Rule(m, w):
    return m.fRV[w] == m.iRV[w] - m.Process[w] * bottle_vol
model.Raw_Volume = Constraint(model.WEEKS, rule = Raw_Volume_Rule)

# constraint 2. the final bottled volume of each week is equal to 95% (after discarding) of the initial bottled volume added to the processed volume

def Bottled_Volume_Rule(m, w):
    return m.fBV[w] == (1 - contaminated_frac) * (m.iBV[w] + m.Process[w] * bottle_vol)
model.Bottled_Volume = Constraint(model.WEEKS, rule = Bottled_Volume_Rule)

# constraint 3. the total stored volume at the end of each week is fBV[w] + iBV[w]

def Total_Volume_Rule(m, w):
    return m.fTV[w] == m.fRV[w] + m.fBV[w]
model.Total_Volume = Constraint(model.WEEKS, rule = Total_Volume_Rule)

# constraint 4. the total weekly demand of the NICUs must be satisfied

def NICU_Demand_Rule(m, w):
    return m.Output[w] >= m.demand[w]
model.NICU_Demand = Constraint(model.WEEKS, rule = NICU_Demand_Rule)

# constraint 4. we cannot output more processed milk than we actually have

def Milk_Restriction_Rule(m, w):
    return m.Output[w] <= m.fRV[w]
model.Milk_Restriction = Constraint(model.WEEKS, rule = Milk_Restriction_Rule)

# constraint 5. the milk bank receives a weekly donation of 35 L of raw milk

def Weekly_Donation_Rule(m, w):
    if w < len(weekly_demand):
        return m.iRV[w + 1] == m.fRV[w] + raw_donation
    return Constraint.Skip
model.Weekly_Donation = Constraint(model.WEEKS, rule = Weekly_Donation_Rule)

# constraint 6. the stored bottled volume at the start of each week depends on how much was shipped out in the previous week

def Stored_Bottles_Rule(m, w):
    if w < len(weekly_demand):
        return m.iBV[w + 1] == m.iBV[w] - m.Output[w] * bottle_vol
    return Constraint.Skip
model.Stored_Bottles = Constraint(model.WEEKS, rule = Stored_Bottles_Rule)

# constraint 7. the total stored volume of processed milk cannot drop below 50 bottles (or 5 L)

def Min_Inventory_Rule(m, w):
    if w < len(weekly_demand):
        return m.iBV[w + 1] >= min_stored_bottles * bottle_vol
    else:
        return m.fBV[w] >= min_stored_bottles * bottle_vol
model.Min_Inventory = Constraint(model.WEEKS, rule = Min_Inventory_Rule)

# constraint 8. we cannot output so much in week w that we do not have enough left to meet the demand of week w + 1

def Proactive_Planning_Rule(m, w):
    if w < len(weekly_demand):
        leftover_vol = m.fTV[w] - m.Output[w] * bottle_vol + (1 - contaminated_frac) * raw_donation
        return leftover_vol >= m.demand[w + 1] * bottle_vol
    return Constraint.Skip
model.Proactive_Planning = Constraint(model.WEEKS, rule = Proactive_Planning_Rule)

# constraint 9. we do not exceed our processing capacity

def Processing_Capacity_Rule(m, w):
    return m.Process[w] <= process_cap
model.Processing_Capacity = Constraint(model.WEEKS, rule = Processing_Capacity_Rule)

# objective function. we seek to maximize the total output over all 12 weeks in the delivery schedule

def Total_Output_Rule(m):
    return sum(m.Output[w] for w in m.WEEKS)
model.Total_Output = Objective(rule = Total_Output_Rule, sense = maximize)

# select a solver and solve the optimization problem

solver = SolverFactory('glpk')
solution = solver.solve(model, tee = True)
print(solution['Solver'])

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp4su3vyk4.glpk.raw --wglp /tmp/tmp5d71a0s0.glpk.glp --cpxlp
 /tmp/tmptoqe28iv.pyomo.lp
Reading problem data from '/tmp/tmptoqe28iv.pyomo.lp'...
117 rows, 82 columns, 242 non-zeros
12 integer variables, none of which are binary
709 lines were read
Writing problem data to '/tmp/tmp5d71a0s0.glpk.glp'...
611 lines were written
GLPK Integer Optimizer 5.0
117 rows, 82 columns, 242 non-zeros
12 integer variables, none of which are binary
Preprocessing...
PROBLEM HAS NO PRIMAL FEASIBLE SOLUTION
Time used:   0.0 secs
Memory used: 0.1 Mb (94292 bytes)
Writing MIP solution to '/tmp/tmp4su3vyk4.glpk.raw'...
208 lines were written

- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.024408817291259766



# Dependencies

Load the code blocks in this section to ensure that all solvers are imported correctly.

In [4]:
# install conda
!pip install -q condacolab
import condacolab
condacolab.install()

# install GLPK
!conda install -c conda-forge glpk --yes

# install IPOPT with conda
!conda install -c conda-forge ipopt --yes

✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [5]:
pip install numpy pandas pyomo gurobipy